## Load in datasets

This will take a minute or two.

In [1]:
from cornell import Cornell

dataset = Cornell()
(train_data, train_labels), (test_data, test_labels) = dataset.load_data()

Using TensorFlow backend.


Also grab the index of encodings to genres.

In [2]:
idx_to_genre = dataset.get_idx_to_genre()

In [3]:
import numpy as np

In [5]:
num_words = 10000
num_genres = len(idx_to_genre)

# From https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/3.5-classifying-movie-reviews.ipynb
def vectorize_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data, num_words)
x_test = vectorize_sequences(test_data, num_words)

y_train = vectorize_sequences(train_labels, num_genres)
y_test = vectorize_sequences(test_labels, num_genres)

In [ ]:
from keras import models, layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(num_words,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(y_train.shape[1], activation='sigmoid'))

In [136]:
from keras import models, layers

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(num_words,)))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_genres, activation='sigmoid'))

In [172]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(num_words,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
# model.add(layers.Dense(16, activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_genres, activation='sigmoid'))

In [173]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [174]:
model.fit(x_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

Train on 415 samples, validate on 47 samples
Epoch 1/20
415/415 [==============================] - 2s 5ms/step - loss: 0.4121 - acc: 0.8301 - val_loss: 0.3170 - val_acc: 0.8768
Epoch 2/20
415/415 [==============================] - 1s 1ms/step - loss: 0.3214 - acc: 0.8718 - val_loss: 0.3034 - val_acc: 0.8839
Epoch 3/20
415/415 [==============================] - 1s 1ms/step - loss: 0.2935 - acc: 0.8809 - val_loss: 0.2809 - val_acc: 0.8777
Epoch 4/20
415/415 [==============================] - 1s 1ms/step - loss: 0.2631 - acc: 0.8922 - val_loss: 0.2662 - val_acc: 0.8910
Epoch 5/20
415/415 [==============================] - 1s 1ms/step - loss: 0.2390 - acc: 0.8988 - val_loss: 0.2569 - val_acc: 0.8945
Epoch 6/20
415/415 [==============================] - 1s 1ms/step - loss: 0.2235 - acc: 0.9074 - val_loss: 0.2622 - val_acc: 0.8830
Epoch 7/20
415/415 [==============================] - 1s 1ms/step - loss: 0.2073 - acc: 0.9146 - val_loss: 0.2505 - val_acc: 0.8980
Epoch 8/20
415/415 [===========

In [175]:
results = model.evaluate(x_test, y_test)
results

155/155 [==============================] - 0s 198us/step


[0.2488469950614437, 0.9083333419215295]

In [153]:
predictions = model.predict(x_test)
predictions[predictions>=0.5] = 1
predictions[predictions<0.5] = 0

array([0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [154]:
entries = len(y_test)
total = num_genres * entries

correct = 0
for i in range(entries):
    for j in range(num_genres):
        correct += int(predictions[i][j] == y_test[i][j])

print(str(correct / total))

0.9048387096774193


In [44]:
predictions = model.predict(y_test)

ValueError: Error when checking input: expected dense_1_input to have shape (10000,) but got array with shape (24,)